In [ ]:
import pandas as pd
import tsfel
import math
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

In [ ]:
soggetti = [1,2,6,7,8,9,11,12,25,26,27,28,29,30,37,38,39,44,
            48,49,52,53,54,56,57,58,59,61,72,76,77,80,81,85,
            86,88,89,91,92,100,110,111,117,118,119,121,122,
            124,125,127,131,132,133,136,138,139,140,144,147,
            156,157,158,159,162,163,164,165,166,168,169,170,
            171,174,175,181,183,184,192,193,194,196,197,198,
            199,200,202,203,204,205,206,207,209,210,211,212,
            213,215,216,217,218,220,221,222,224,225,229,230,
            231,232,237,238,241,244,245,246,256,257,260,263,
            264,270,271,274,275,277,278,279,280,284,285,286,
            287,288,290,291,292,293,294,295,296,297,310,311,
            317,322,323,324,326,327,329,330,331,332,333,334,
            336,337,338,339,341,345,346,347,348,349,350,351,
            354,356,357,358,359,360,361,365,371,376,377,387,
            388,389,391,393,394,404,409,410,412,413,433]

quadri = [1,2,4,6,7,9,10,11,13,14,15,16,18,20,21,22,23,45,47]

folder_path = 'coords_filtered/'

dfeat = pd.read_csv('Extracted_Features.csv')
# Imposta le prime due colonne come indice
dfeat.set_index(['ID', 'PAINTING'], inplace=True)

In [ ]:

def calculate_fixations(df, max_dispersion=15, min_duration_points=4):
    fixations = []
    start_index = 0

    while start_index < len(df):
        # Inizializza la fixation corrente
        current_fixation = [df.iloc[start_index]]
        i = start_index + 1

        while i < len(df):
            # Aggiungi il punto corrente alla fixation se la dispersione è inferiore alla soglia
            current_fixation.append(df.iloc[i])
            fixation_df = pd.DataFrame(current_fixation)
            
            # Controlla se la dispersione corrente supera la soglia
            if (fixation_df['x'].max() - fixation_df['x'].min() > max_dispersion or
                fixation_df['y'].max() - fixation_df['y'].min() > max_dispersion):
                # Rimuovi l'ultimo punto aggiunto se supera la dispersione
                current_fixation.pop()
                break
            i += 1

        # Controlla se la fixation ha una durata sufficiente
        if len(current_fixation) >= min_duration_points:
            fixation_x = pd.DataFrame(current_fixation)['x'].mean()
            fixation_y = pd.DataFrame(current_fixation)['y'].mean()
            duration = len(current_fixation) * 250  # ogni punto rappresenta 250 ms
            fixations.append((fixation_x, fixation_y, duration))

        # Aggiorna l'indice di partenza per la prossima fixation
        start_index = i if i > start_index + 1 else start_index + 1

    return fixations

In [ ]:
def analyze_list(data):
    df = pd.DataFrame(data, columns=['x', 'y', 'duration'])
    
    if len(df) < 2:
        return None  # Non abbastanza dati per clusterizzare
    
    max_k = int(np.sqrt(len(df)))  # Calcolo di un massimo ragionevole per k
    silhouette_scores = {}
    
    for k in range(2, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(df[['x', 'y']])
        score = silhouette_score(df[['x', 'y']], labels)
        silhouette_scores[k] = score
        
    if not silhouette_scores:
        return None  # Impossibile calcolare un punteggio significativo
    
    optimal_k = max(silhouette_scores, key=silhouette_scores.get)
    
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    df['cluster'] = kmeans.fit_predict(df[['x', 'y']])
    
    # Calcolo delle statistiche per ogni cluster
    cluster_stats = df.groupby('cluster')['duration'].agg(['sum', 'count']).reset_index()
    cluster_stats.columns = ['Cluster ID', 'Total Duration', 'Number of Points']
    
    # Trova il cluster con il maggior numero di elementi
    max_cluster = cluster_stats.loc[cluster_stats['Number of Points'].idxmax()]
    
    return {
        'max_similar_fixations': max_cluster['Number of Points'],
        'max_durata_similar_fixations': max_cluster['Total Duration'],
        'n_aree_interesse': optimal_k
    }

In [ ]:
df = pd.DataFrame()

data = []

for s in soggetti:
    try:
        dp = pd.read_csv(f'ProcessingCSV/SoggettoQuadro/PupilData/S{s}_Qblank.csv')
        max_diam_blank = dp['DIAM'].max()
    except:
        print(f'blank di {s} non presente')
    for q in quadri:
        try:
            #Carico il dataframe con le coordinate appartenenti al solo quadro
            dt = pd.read_csv(f'{folder_path}S{s}_Q{q}.csv')
            punti_nel_quadro = dt.shape[0] #Il numero di righe sono il numero coordinate appartenenti al quadro

            '''Per ogni soggetto x quadro mi calcolo le fixations e le loro coordinate in un dataframe
            Per ogni fixation avrò x, y, durata.
            Userò la deviazione standard delle coordinate per definire il threshold per la dispersione delle coordinate
            Per il threshold della durata userò 2000ms, 8 istanze contigue, ovvero 2 secondi'''
            #Calcolo le fixations sul dataset filtered
            # Calcolo delle deviazioni standard per x e y
            rng_x = dt['x'].max() - dt['x'].min()
            rng_y = dt['y'].max() - dt['y'].min()

            dispersion_threshold = math.sqrt((rng_x/20)**2 +(rng_y/20)**2)/2
            duration_threshold = 8

            fixations = calculate_fixations(dt, max_dispersion=dispersion_threshold, min_duration_points=duration_threshold)

            result = analyze_list(fixations)

            max_similar_fixations, max_durata_similar_fixations, n_aree_interesse = result['max_similar_fixations'], result['max_durata_similar_fixations'], result['n_aree_interesse']

            #Copio le features estratte nel dataframe Extrated_Features per diametro pupilla sulla finestra full
            mean_diam_pupilla = dfeat.loc[(s,q), 'mean_diametro_full']
            min_diam_pupilla = dfeat.loc[(s,q), 'min_diametro_full']
            max_diam_pupilla = dfeat.loc[(s,q), 'max_diametro_full']
            std_diam_pupilla = dfeat.loc[(s,q), 'std_diametro_full']
            diff_max_quadro_blank = dfeat.loc[(s,q), 'max_diametro_full'] - max_diam_blank

            istanza = {
                'ID' : s,
                'PAINTING' : q,
                'PUNTI_NEL_QUADRO' : punti_nel_quadro,
                'FIXATIONS' : len(fixations),
                'mean_diam_pupilla' : mean_diam_pupilla,
                'min_diam_pupilla' : min_diam_pupilla,
                'max_diam_pupilla' : max_diam_pupilla,
                'std_diam_pupilla' : std_diam_pupilla,
                'diff_max_quadro_blank' : diff_max_quadro_blank,
                'max_similar_fixations' : max_similar_fixations,
                'max_durata_similar_fixations' : max_durata_similar_fixations,
                'n_aree_interesse' : n_aree_interesse
            }

            data.append(istanza)
        except:
            print(f'Errore in soggetto {s} quadro {q}')
df = pd.concat([pd.DataFrame([d]) for d in data], ignore_index=True)
print(df)

In [ ]:
def analyze_list(data):
    df = pd.DataFrame(data, columns=['x', 'y', 'duration'])
    
    if len(df) < 2:
        return None  # Non abbastanza dati per clusterizzare
    
    max_k = int(np.sqrt(len(df)))  # Calcolo di un massimo ragionevole per k
    silhouette_scores = {}
    
    for k in range(2, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(df[['x', 'y']])
        score = silhouette_score(df[['x', 'y']], labels)
        silhouette_scores[k] = score
        
    if not silhouette_scores:
        return None  # Impossibile calcolare un punteggio significativo
    
    optimal_k = max(silhouette_scores, key=silhouette_scores.get)
    
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    df['cluster'] = kmeans.fit_predict(df[['x', 'y']])
    
    cluster_info = df.groupby('cluster')['duration'].agg(['sum', 'count']).reset_index()
    cluster_info.columns = ['Cluster ID', 'Total Duration', 'Number of Points']
    
    return cluster_info, optimal_k

# Applicazione su tutte le liste
results = [analyze_list(fixations)]  # Filtraggio delle liste vuote

# Visualizzazione di un esempio di risultato (assicurarsi che ci siano risultati)
if results:
    print(results[0])  # Mostra il risultato per la prima lista
else:
    print("Nessun risultato disponibile per la visualizzazione")

In [ ]:
df.to_csv('Extracted_Features_2.csv', index=False)

In [ ]:
df = pd.read_csv('ProcessingCSV/SoggettoQuadro/PupilData/S1_Qblank.csv')

In [ ]:
df

In [ ]:
ists = []
# Carica la configurazione di TSFEL
cfg = tsfel.get_features_by_domain()

# DataFrame finale per raccogliere tutte le features
all_features = pd.DataFrame()
for s in soggetti:
    for q in quadri:
        try:
            df = pd.read_csv(f'ProcessingCSV/SoggettoQuadro/PupilData/S{s}_Q{q}.csv')

            ts = df['DIAM']
                
            # Estrazione delle features
            features = tsfel.time_series_features_extractor(cfg, ts, verbose=0)
                
            istanza={
                'ID' : s,
                'PAINTING' : q
            }

            ists.append(istanza)
                
            # Appendi al DataFrame finale
            all_features = pd.concat([all_features, features], ignore_index=True)
        except Exception as e:
            print(f'Errore in soggetto {s} quadro {q}')
            print(e)

istanze = pd.concat([pd.DataFrame([i]) for i in ists], ignore_index=True)

if len(all_features) == len(istanze):
    # Concatena le colonne
    result = pd.concat([istanze, all_features], axis=1)
else:
    print("I DataFrame non hanno la stessa lunghezza e non possono essere concatenati.")

result


In [ ]:
result.to_csv('Diam_Extracted_Features_TimeSerie.csv', index=False)